# Add Input Columns
The original models did not include the input columns. This
notebook will add 'input_col' to the data_def dict of all
models previously created

In [ ]:
data_cols: dict[str, list[str]] = {}


In [ ]:
from glob import glob
import json
import os

import pandas as pd

from train_test import load_csv, infer_feature_cols

data_filenames: dict[tuple[str, str], str] = {
    ("mlb", "h"): "mlb_hitter.csv",
    ("mlb", "p"): "mlb_pitcher.csv",
    ("nba", "dk"): "nba_player.csv",
    ("nfl", "def"): "nfl_team.csv",
    ("nfl", "wrte"): "nfl_wrte.csv",
    ("nfl", "qb"): "nfl_qb.csv",
    ("nfl", "rb"): "nfl_rb.csv",
}

# iterate through all .model files
for model_filepath in glob("*model"):
    print(f"Adding cols to '{model_filepath}")

    # load model json
    with open(model_filepath, "r") as f_:
        model_json = json.load(f_)
    include_position = (
        model_json["training_data_def"]["include_pos"]
        if model_json["training_data_def"]["target"][1] == "P"
        else None
    )
    print(f"{model_json['name']=} {include_position}")

    # load data used to create the model
    sport: str
    extra: str
    sport, extra = model_json["name"].lower().split("-", 3)[:2]
    data_filename = os.path.join(
        os.environ["FANTASY_HOME"],
        data_filenames.get((sport, extra), f"{sport}_{extra}.csv"),
    )
    print(f"Getting cols for {sport=} {extra=} in '{data_filename}'")
    if data_filename not in data_cols:
        df = load_csv(
            data_filename,
            include_position=include_position,
        )

        cols = df.columns.to_list()
        data_cols[data_filename] = cols
    else:
        cols = data_cols[data_filename]
    feature_cols = infer_feature_cols(cols, include_position)
    print(f"{len(feature_cols)} input columns for '{data_filename}'", feature_cols)

    # update model json with the column names
    model_json["training_data_def"]["input_cols"] = feature_cols

    # save updated json
    with open(model_filepath, "w") as f_:
        json.dump(model_json, f_, indent="\t")

    print("Wrote file\n--------------------------------------\n")
